In [1]:
import json

categories=(
    "All_Beauty",
    "Amazon_Fashion",
    "Appliances",
    "Arts_Crafts_and_Sewing",
    "Automotive",
    "Baby_Products",
    "Beauty_and_Personal_Care",
    "Books",
    "CDs_and_Vinyl",
    "Cell_Phones_and_Accessories",
    "Clothing_Shoes_and_Jewelry",
    "Digital_Music",
    "Electronics",
    "Gift_Cards",
    "Grocery_and_Gourmet_Food",
    "Handmade_Products",
    "Health_and_Household",
    "Health_and_Personal_Care",
    "Home_and_Kitchen",
    "Industrial_and_Scientific",
    "Kindle_Store",
    "Magazine_Subscriptions",
    "Movies_and_TV",
    "Musical_Instruments",
    "Office_Products",
    "Patio_Lawn_and_Garden",
    "Pet_Supplies",
    "Software",
    "Sports_and_Outdoors",
    "Subscription_Boxes",
    "Tools_and_Home_Improvement",
    "Toys_and_Games",
    "Video_Games",
)

## examples

In [2]:
import os
import subprocess

results = []
line_number = 100
for c in categories:
    results.append(
                subprocess.run(f"sed -n '{line_number}{{p;q;}}' {c + '.jsonl'}", shell=True, check=True, stdout=subprocess.PIPE, stderr=subprocess.PIPE, text=True).stdout)

In [6]:
json.loads(results[0])

{'main_category': 'All Beauty',
 'title': 'QTHAIR Brazilian Body Wave Human Hair Bundles 16 18 20 300g Natural Black Brazilian Weave Human Hair Bundles Brazilian Virgin Hair For African Americans Women',
 'average_rating': 5.0,
 'rating_number': 1,
 'features': [],
 'description': [],
 'price': None,
 'images': [{'thumb': 'https://m.media-amazon.com/images/I/51iHyiXwN9L._SS40_.jpg',
   'large': 'https://m.media-amazon.com/images/I/51iHyiXwN9L.jpg',
   'variant': 'MAIN',
   'hi_res': 'https://m.media-amazon.com/images/I/618DquZP6cL._SL1050_.jpg'},
  {'thumb': 'https://m.media-amazon.com/images/I/51gyOtzY8PL._SS40_.jpg',
   'large': 'https://m.media-amazon.com/images/I/51gyOtzY8PL.jpg',
   'variant': 'PT01',
   'hi_res': 'https://m.media-amazon.com/images/I/71yT6ckJNGL._SL1050_.jpg'},
  {'thumb': 'https://m.media-amazon.com/images/I/61PbObTN6FL._SS40_.jpg',
   'large': 'https://m.media-amazon.com/images/I/61PbObTN6FL.jpg',
   'variant': 'PT02',
   'hi_res': 'https://m.media-amazon.com/im

In [11]:
for r in results:
    r = json.loads(r)
    print(f"title: {r['title']}")
    print(f"features: {r['features']}")
    print(f"description: {r['description']}")
    print("------------------------------------")
    # print({
    #     'title': r['title'],
    #     'features': r['features'],
    #     'description': r['description']
    # })

title: QTHAIR Brazilian Body Wave Human Hair Bundles 16 18 20 300g Natural Black Brazilian Weave Human Hair Bundles Brazilian Virgin Hair For African Americans Women
features: []
description: []
------------------------------------
title: Alchemy Gothic England - Metallica: Justice for All Pin Badge
features: []
description: []
------------------------------------
title: Samsung DG94-00935A Range Surface Burner Valve Genuine Original Equipment Manufacturer (OEM) Part Pink
features: ['Samsung ASSY VALVE AUX;FX710BGS, (DG94-00935A)']
description: ['Samsung Range/Stove/Oven Surface Burner Valve.', 'Genuine OEM Part # DG94-00935A.']
------------------------------------
title: Hydrographic Film Burl Wood Grain Hydro Graphic Water Transfer Film Hydro Dipping Dip Demon Wizard Ape
features: ['NEW PREMIUM HYDROGRAPHIC WATER TRANSFER FILM RECOMMENDED BASE COAT : WHITE', "HIGHEST QUAILITY INKS AND MATERIALS used to insure CONSTANT and VIBRANT DIP'S / TRANSFERS", 'LABELED FOR EASE OF USE / PROFESS

## length of tokens

In [12]:
from transformers import AutoTokenizer

tokenizer = AutoTokenizer.from_pretrained(
                '/data1/home/xingmei/GRE/e5-base-v2',
                use_fast=True,
                revision='main',
                trust_remote_code=True,
            )

/data1/home/xingmei/.conda/envs/rec/lib/python3.11/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [15]:
len_of_title = 0
len_of_feature = 0
len_of_description = 0
cnt = 0

for c in categories:
    with open(c + '.jsonl', 'r') as f:
        for line in f:
            metadata = json.loads(line.strip())
            len_of_title += len(tokenizer(metadata['title'])['input_ids'])
            len_of_feature += len(tokenizer(' '.join(metadata['features']))['input_ids'])
            len_of_description += len(tokenizer(' '.join(metadata['description']))['input_ids'])
            cnt += 1

In [ ]:
print(f"Average lenth of title's tokens: {len_of_title / cnt}")
print(f"Average lenth of feature's tokens: {len_of_feature / cnt}")
print(f"Average lenth of description's tokens: {len_of_description / cnt}")

## length of behaviors

In [ ]:
import pandas as pd

for c in categories:
    df = pd.read_csv(f'../pureID_inter/{c}.csv', header=0, sep=',')
    df.drop_duplicates(inplace=True)
    len_of_bh = df.groupby('user_id').count()['parent_asin'].mean()
    print(f"{c}:\t\t\t{len_of_bh}")